In [ ]:
import os
import numpy as np
import pandas as pd
import time
import copy
import datetime
from datetime import datetime

# 忽略警告訊息
import warnings
warnings.filterwarnings( 'ignore' )

###  輸入相關資訊

In [ ]:
""" 輸入評估日 """
valuation_date = '2018-06-30'


""" 輸入資料儲存路徑 """
# 前期產出的資料路徑
dir_data_1 = '/Users/YenLin/Desktop/PAA'    

# 當期保單、保費及賠款資料路徑
dir_data_2 = '/Users/YenLin/Desktop/PAA/Data/'


""" 輸入當期保單及保費資料檔名 """
policy_data_name = '保單資料_2018Q2.txt'
premium_data_name = '保費資料_2018Q2.txt'
loss_data_name = '賠款資料_2018Q2.txt'


# 評估所屬年度及季別
valuation_year = int( valuation_date[ 0:4 ] )
if ( valuation_date[ 5:7 ] == '03' ) : 
    Quarter = 'Q1'
    previous_Quarter = 'Q4'
    previous_valuation_year = valuation_year - 1 
elif ( valuation_date[ 5:7 ] == '06' ) : 
    Quarter = 'Q2'
    previous_Quarter = 'Q1'
    previous_valuation_year = valuation_year 
elif ( valuation_date[ 5:7 ] == '09' ) : 
    Quarter = 'Q3'  
    previous_Quarter = 'Q2'
    previous_valuation_year = valuation_year 
else : 
    Quarter = 'Q4'  
    previous_Quarter = 'Q3'
    previous_valuation_year = valuation_year 
    
# 前期產出的資料檔名
previous_data_name = str( previous_valuation_year ) + '_' + Quarter + '_data.txt' 
previous_statement_data_name = str( previous_valuation_year ) + '_' + Quarter + '_statement_data.txt' 
Position_table_name = str( previous_valuation_year ) + '_' + Quarter + '_Position_table.txt'
Performance_table_name = str( previous_valuation_year ) + '_' + Quarter + '_Performance_table.txt'


##### 讀取資料

In [ ]:
# 前期產出的資料
previous_data = os.path.join( dir_data_1, previous_data_name )
previous_statement_data = os.path.join( dir_data_1, previous_statement_data_name )
Position_table = os.path.join( dir_data_1, Position_table_name )
Performance_table = os.path.join( dir_data_1, Performance_table_name )

# 當期保單及保費資料
policy_data = os.path.join( dir_data_2, policy_data_name )
premium_data = os.path.join( dir_data_2, premium_data_name )
loss_data = os.path.join( dir_data_2, loss_data_name )


print( 'Path of Previous Data: %s' % ( previous_data ) )
print( 'Path of Previous Statement Data: %s' % ( previous_statement_data ) )
print( 'Path of Previous Position Table: %s' % ( Position_table ) )
print( 'Path of Previous Performance Table: %s' % ( Performance_table ) )
print( 'Path of Policy Data: %s' % ( policy_data ) )
print( 'Path of Premium Data: %s' % ( premium_data ) )
print( 'Path of Loss Data: %s' % ( loss_data ) )


# 資料讀取
Previous_data = pd.read_csv( previous_data, encoding = 'utf-8' )
Previous_statement_data = pd.read_csv( previous_statement_data, encoding = 'utf-8' )
Position_table = pd.read_csv( Position_table, encoding = 'utf-8' )
Performance_table = pd.read_csv( Performance_table, encoding = 'utf-8' ) 
Policy_data = pd.read_csv( policy_data, encoding = 'utf-8' )
Premium_data = pd.read_csv( premium_data, encoding = 'utf-8' )
Loss_data = pd.read_csv( loss_data, encoding = 'utf-8' )

##### 保單及保費資料格式調整
- 時間欄位  
- 百分比轉浮點小數

In [ ]:
Policy_date_col = [ '簽單日', '保費到期日', '保險起日', '保險迄日' ]
Policy_percentage_col = [ '佣金率' ]
Premium_date_col = [ '保費收取日', '佣金支付日' ]

for col in Policy_date_col :
    Policy_data[ col ] = Policy_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

for col in Policy_percentage_col :  
    Policy_data[ col ] = Policy_data[ col ].str.strip( '%' ).astype( float ) / 100 

for col in Premium_date_col :
    Premium_data[ col ] = Premium_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 
    
    
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當期之保單資料筆數 = ' + str( Policy_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當期之保費資料筆數 = ' + str( Premium_data.shape[0] ) + '\n' )


##### 檢視當期保單及保費資料（ 前15筆 ）

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當期之保單資料 : ' ) 
Policy_data.head( 15 )

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當期之保費資料 : ' ) 
Premium_data.head( 15 )

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當期之賠款資料 : ' ) 
Loss_data.head( 15 )

##### 合併保單及保費資料
- 當期合併
- 當期與前期合併
- 更新 保費 及 佣金 相關欄位 ( ex: 金額及日期 )
- 保單除列 ？？？

In [ ]:
cols_name = [ '保批單號碼', 'Group', '簽單日', '保費到期日', '保險起日', '保險迄日', '簽單保費', '佣金率', '保費收取日', '保費收取金額', '佣金支付日', '佣金支付金額' ]

# 當期合併
current_data = pd.merge( Policy_data, Premium_data, on = [ '保批單號碼' ], how = 'left' )
current_data = current_data[ cols_name ]                # 調整欄位順序
current_data_temp = copy.deepcopy( current_data )  # 複製資料

# 當期與前期資料合併
Data = pd.concat( [ Previous_data, current_data ], axis = 0 )

# 更新前期尚未收取保費的保單
Data.set_index( '保批單號碼', inplace = True )
Premium_data.set_index( '保批單號碼', inplace = True )
Data.update( Premium_data ) 


print( '合併且更新後，資料總比數 = ' + str( Data.shape[0] ) + '\n' )
print( '檢視合併且更新後資料 (前15筆) : ' )
Data.head( 15 )

##### 計算每張保單的「 保期(月) 」及「 當期已過月數 」

In [ ]:
# 定義計算月數函數
def months( data, evaluate_date ) :
    start_year = data[ '保險起日' ].apply( lambda X : X.year )                             # 年度 of 保險起日
    end_year = data[ '保險迄日' ].apply( lambda X : X.year )                              # 年度 of 保險迄日
    start_month = data[ '保險起日' ].apply( lambda X : X.month )                       # 月份 of 保險起日
    end_month = data[ '保險迄日' ].apply( lambda X : X.month )                        # 月份 of 保險迄日
    evaluate_date = datetime.strptime( evaluate_date, '%Y-%m-%d' ).date( )    # 評估日
    evaluate_year = evaluate_date.year                                                          # 評估日所屬年度
    evaluate_month = evaluate_date.month                                                    # 評估日所屬月份
    
    if ( evaluate_month == 3 ) : 
        q_start_date = str( evaluate_year ) + '-' + '01-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期   
    elif ( evaluate_month == 6 ) :
        q_start_date = str( evaluate_year ) + '-' + '04-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期
    elif ( evaluate_month == 9 ) :
        q_start_date = str( evaluate_year ) + '-' + '07-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期
    else :  
        q_start_date = str( evaluate_year ) + '-' + '10-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期

    # 計算「 保期(月) 」
    data[ '保期(月)' ] = ( end_year - start_year )*12 + ( end_month - start_month + 1 )
    
    # 計算「 當期已經過月數 」
    data[ '當期已經過月數' ] = np.nan
    for i in np.arange( data.shape[0] ) : 
        first_date = max( q_start_date,  data[ '保險起日' ][ i ] )
        least_date = min( evaluate_date, data[ '保險迄日' ][ i ] ) 
        first_year = first_date.year
        first_month = first_date.month
        least_year = least_date.year
        least_month = least_date.month
        data[ '當期已經過月數' ][ i ] = max( ( least_year - first_year )*12 + ( least_month - first_month + 1 ), 0 )

    return data.head( 15 )


In [ ]:
months( Data, evaluate_date )

##### 填補及計算下列項目
- 填補「 期初LRC 」               
- 填補「 當期已收保費 」        
- 填補「 當期已付佣金 」       
- 計算「 當期認列之合約收入 」
- 計算「 取得成本攤銷 」         
- 計算「 期末LRC 」               
- 計算「 累積已收保費 」         
- 計算「 累積已付佣金 」         

In [ ]:
Data.reset_index( level = '保批單號碼', inplace = True )

# 前期資料的「 保批單號碼 」
Previous_Policy_NO = list( Previous_statement_data[ '保批單號碼' ] )

# 填補「 期初LRC 」
Data[ '期初LRC' ] = np.nan  
for i in np.arange( Data.shape[0] ) :   
    Policy_NO = Q12_data[ '保批單號碼' ][ i ]
    if ( Policy_NO in Previous_Policy_NO ):   
        Data[ '期初LRC' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '期末LRC'  ]
    else : 
        Data[ '期初LRC' ][ i ] = 0
        
# 填補「 當期已收保費 」 及 「 當期已付佣金 」
Data[ '當期已收保費' ] = np.nan  
Data[ '當期已付佣金' ] = np.nan  
for i in np.arange( Data.shape[0] ) :   
    Policy_NO = Data[ '保批單號碼' ][ i ]
    if Data[ '保費收取日' ][ i ] >= q_start_date :
        Data[ '當期已收保費' ][ i ] = Data[ '保費收取金額' ][ i ] 
        Data[ '當期已付佣金' ][ i ] = Data[ '佣金支付金額' ][ i ] 
    else : 
        Data[ '當期已收保費' ][ i ] = 0
        Data[ '當期已付佣金' ][ i ] = 0        
        
# 計算「 當期認列之合約收入 」、「 取得成本攤銷 」及「 期末LRC 」
Data[ '當期認列之合約收入' ] =  Data.apply( lambda X : X[ '簽單保費' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )
Data[ '取得成本攤銷' ] = Data.apply( lambda X : X[ '簽單保費' ] * X[ '佣金率' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )
Data[ '期末LRC' ] =  Data.apply( lambda X : X[ '期初LRC' ] + X[ '當期已收保費' ] - X[ '當期已付佣金' ] - X[ '當期認列之合約收入' ] + X[ '取得成本攤銷' ] , axis=1 )

# 計算「 累積已收保費 」及「 累積已付佣金 」
Data[ '累積已收保費' ] = np.nan
Data[ '累積已付佣金' ] = np.nan
for i in np.arange( Data.shape[0] ) :   
    Policy_NO = Data[ '保批單號碼' ][ i ]
    if ( Data[ '保批單號碼' ][ i ] in Previous_Policy_NO ) : 
        Data[ '累積已收保費' ][ i ] = previous_statement_data.loc[ previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '累積已收保費'  ] + Data.loc[ previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '當期已收保費'  ]
        Data[ '累積已付佣金' ][ i ] = previous_statement_data.loc[ previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '累積已付佣金'  ] + Data.loc[ previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '當期已付佣金'  ]
    else : 
        Data[ '累積已收保費' ][ i ] = Data[ '當期已收保費' ][ i ] 
        Data[ '累積已付佣金' ][ i ] = Data[ '當期已付佣金' ][ i ] 

        
Data.head( 15 )

##### 填報「 財務狀況表 」及「 財務績效表（當季）」

In [ ]:
item = str( valuation_year ) + ' ' + Quarter 
previous_item = str( previous_valuation_year ) + ' ' + previous_Quarter 

select_col = [ '保批單號碼',  '期初LRC', '當期已收保費', '當期已付佣金', '期末LRC', '當期認列之合約收入', '取得成本攤銷', '累積已收保費', '累積已付佣金' ]
statement_data = Data[ select_col ]

In [ ]:
# 「 財務狀況表 」
Position_table.loc[ item, : ] = np.nan

# 「 現金 」(尚未考慮當期賠款！！)
Position_table.loc[ item, '現金']  = Position_table.loc[ previous_item, '現金'] + statement_data[ '當期已收保費' ].sum( ) - statement_data[ '當期已付佣金' ].sum( )

# 「 合約資產 」或「 合約負債 」
LRC = statement_data[ '期末LRC' ].sum( )
if LRC > 0 :
    Position_table.loc[ item, '合約資產' ] = 0
    Position_table.loc[ item, '合約負債' ] = LRC
elif LRC < 0 :
    Position_table.loc[ item, '合約資產' ] = -LRC
    Position_table.loc[ item, '合約負債' ] = 0
else : 
    Position_table.loc[ item, '合約資產' ] = 0 
    Position_table.loc[ item, '合約負債' ] = 0

Position_table[ '業主權益' ] = Position_table.apply( lambda X : X[ '現金' ] + X[ '合約資產' ] - X[ '合約負債' ], axis = 1 )

print( str( valuation_year ) + ' ' + str( Quarter ) + ' 財務狀況表：' )
Position_table

In [ ]:
# 「 財務績效表（當季）」
Performance_table.loc[ item, : ] = np.nan

Performance_table.loc[ item, '保險合約收入' ] = statement_data[ '當期認列之合約收入' ].sum( )
Performance_table.loc[ item, '取得成本攤銷' ] = statement_data[ '取得成本攤銷' ].sum( )
Performance_table.loc[ item, '已發生賠款' ] = 0 # (尚未考慮當期賠款！！)
Performance_table[ '保險損益' ] = Performance_table.apply( lambda X : X[ '保險合約收入' ] - X[ '已發生賠款' ] - X[ '取得成本攤銷' ], axis=1 )

print( str( valuation_year ) + ' ' + str( Quarter ) + ' 財務績效表(當季)：' )
Performance_table

##### 儲存檔案 

In [ ]:
current_data_temp.to_csv( str( item ) + '_data.txt', index = True, encoding = 'utf-8' ) 
statement_data.to_csv( str( item ) + '_statement_data.txt', index = True, encoding = 'utf-8' ) 
Position_table.to_csv( str( item ) + '_Position_table.txt', index = True, encoding = 'utf-8' ) 
Performance_table.to_csv( str( item ) + '_Performance_table.txt', index = True, encoding = 'utf-8' ) 